# MWS Secondary Target Selection

## MWS_CALIB_SEGUE

Maintainers:
  - Ting Li (selection/catalog)
  - Andrew Cooper (secondary target code)

In [2]:
ls ../raw

sdss9_xm_gaia_4DESI_v0.fits


{ DESCRIPTION NEEDED }

The selection criteria are:

???

In [3]:
pwd

'/global/projecta/projectdirs/desi/external/mws/sv/MWS_CALIB_SEGUE/1/code'

In [4]:
import os
import numpy as np
from astropy.table import Table, Column
import astropy.io.fits as fits

In [5]:
target_class   = 'MWS_CALIB_SEGUE'
input_filename = 'sdss9_xm_gaia_4DESI_v0.fits'

REF_CAT   = 'G'
REF_EPOCH = '2015.5'

In [6]:
# Relative paths
input_path  = os.path.join('..','raw',input_filename)

# Separate bright and faint output
output_path = os.path.join('..','indata','{}.fits'.format(target_class))

output_dir  = os.path.dirname(output_path)
if not os.path.exists(output_dir):
    print('Making output directory: {}'.format(os.path.abspath(output_dir)))
    os.makedirs(dir)

In [7]:
print('Reading input: {}'.format(os.path.abspath(input_path)))
d = Table.read(input_path)
print('Read {:d} rows'.format(len(d)))

Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_CALIB_SEGUE/1/raw/sdss9_xm_gaia_4DESI_v0.fits
Read 950007 rows


In [8]:
print('First 3 rows:')
print(d[0:3])
print(d.colnames)

First 3 rows:
     source_id             objid        ... phot_rp_mean_mag   ebv   
------------------- ------------------- ... ---------------- --------
1962465291641071360 1237672025706857364 ...              nan 0.320162
1962465287342442880 1237672025706857364 ...          17.5963 0.320232
1961208137534314880 1237671938749893662 ...          14.4844 0.330373
['source_id', 'objid', 'specobjid', 'ra', 'dec', 'pmra', 'pmdec', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'ebv']


In [9]:
BRIGHT_TIME_BRIGHT_LIMIT = 16.0

finite_mag                 = np.isfinite(d['phot_g_mean_mag'])
brighter_than_bright_limit = (d['phot_g_mean_mag'] < BRIGHT_TIME_BRIGHT_LIMIT) & finite_mag
fainter_than_bright_limit  = (d['phot_g_mean_mag'] > BRIGHT_TIME_BRIGHT_LIMIT) & finite_mag

print('Undefined mags for {:d} targets'.format((~finite_mag).sum()))
print('Mag range: {:5.2f} < G < {:5.2f}'.format(d['phot_g_mean_mag'][finite_mag].min() ,d['phot_g_mean_mag'][finite_mag].max()))
print('Targets brighter than G = 16: {}/{} ({:3.2%})'.format(brighter_than_bright_limit.sum(),
                                                             len(d),
                                                             brighter_than_bright_limit.sum()/len(d)))

bright_time_targets = fainter_than_bright_limit

Undefined mags for 0 targets
Mag range:  6.19 < G < 21.50
Targets brighter than G = 16: 66035/950007 (6.95%)


In [10]:
import matplotlib.pyplot as pl
pl.hist(d['phot_g_mean_mag'][finite_mag],bins=np.arange(0,22,0.5));
pl.xlabel('G');
pl.axvline(BRIGHT_TIME_BRIGHT_LIMIT,c='r',alpha=0.5);

Adjust to secondary target data model:

In [11]:
output = d['ra','dec','source_id'][bright_time_targets]
output.rename_columns(['ra','dec','source_id'],['RA','DEC','REF_ID'])
output.add_column(Column(np.repeat(False,len(output)),'OVERRIDE'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMRA'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMDEC'))
output.add_column(Column(np.repeat(REF_CAT,len(output)),'REF_CAT'))
output.add_column(Column(np.repeat(REF_EPOCH,len(output)),'REF_EPOCH'))

Add header:

In [12]:
output.meta['VRAWDATA'] = int(os.path.abspath(input_path).split(os.path.sep)[-3])

In [13]:
output[0:3]

RA,DEC,REF_ID,OVERRIDE,PMRA,PMDEC,REF_CAT,REF_EPOCH
float64,float64,int64,bool,float32,float32,str1,str6
331.005265786112,43.8489545073689,1962465291641071360,False,0.0,0.0,G,2015.5
331.005099021819,43.8488248802653,1962465287342442880,False,0.0,0.0,G,2015.5
329.606204063371,43.9767787247824,1961208137529073024,False,0.0,0.0,G,2015.5


Write output:

In [14]:
print('Writing {:d} rows of output to {}'.format(len(output),os.path.abspath(output_path)))
output.write(output_path,overwrite=True)

Writing 883965 rows of output to /global/projecta/projectdirs/desi/external/mws/sv/MWS_CALIB_SEGUE/1/indata/MWS_CALIB_SEGUE.fits
